In [16]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 4000)

In [25]:
ticker_data = pd.read_csv('/Users/liuda/Local/data/trading/data/stocks/ticker/history/AMZN.csv')
ticker_data['Adj Open'] = ticker_data['Open'] + ticker_data['Adj Close'] - ticker_data['Close']
ticker_data['Adj High'] = ticker_data['High'] + ticker_data['Adj Close'] - ticker_data['Close']
ticker_data['Adj Low'] = ticker_data['Low'] + ticker_data['Adj Close'] - ticker_data['Close']

ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
ticker_data.set_index('Date', inplace=True)

In [26]:
ticker_data_fill_30_day = ticker_data.copy(deep=True)
future_dates = pd.date_range(start=ticker_data_fill_30_day.index[-1] + pd.Timedelta(days=1), periods=30)
ticker_data_fill_30_day = ticker_data_fill_30_day.reindex(ticker_data_fill_30_day.index.union(future_dates))
ticker_data_fill_30_day = ticker_data_fill_30_day.ffill()
ticker_data_fill_30_day.rename_axis('Date', inplace=True)   # index列重命名为Date，后续Join要使用

In [27]:
resampled_data = ticker_data_fill_30_day.resample(   # 从fill 30 day 采样，避免尾部值缺失
    'W', label="left", offset="1D"
).agg(
    {
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Adj High": "max",
        "Adj Open": "first",
        "Adj Low": "min",
        "Adj Close": "last",
        "Volume": "sum",
    }
)

In [28]:
resampled_data

,Open,High,Low,Adj High,Adj Open,Adj Low,Adj Close,Volume
Date,,,,,,,,
2010-03-14,6.620500,6.685500,6.483000,6.685500,6.620500,6.483000,6.517500,365702000.0
2010-03-21,6.510000,6.849500,6.360000,6.849500,6.510000,6.360000,6.753000,741364000.0
2010-03-28,6.768500,6.909500,6.559000,6.909500,6.768500,6.559000,6.590500,484120000.0
2010-04-04,6.642500,7.066500,6.539000,7.066500,6.642500,6.539000,7.003000,768248000.0
2010-04-11,7.000000,7.358500,6.956000,7.358500,7.000000,6.956000,7.108500,687150000.0
...,...,...,...,...,...,...,...,...
2025-03-09,195.600006,201.520004,190.850006,201.520004,195.600006,190.850006,198.889999,334314800.0
2025-03-16,200.720001,201.520004,195.289993,201.520004,200.720001,195.289993,198.889999,305145400.0
2025-03-23,200.720001,201.520004,195.289993,201.520004,200.720001,195.289993,198.889999,305145400.0


In [29]:
ticker_data = ticker_data.merge(resampled_data, on='Date', how='outer', suffixes=('', '_y'))

In [30]:
ticker_data

,Adj Close,Close,High,Low,Open,Volume,Adj Open,Adj High,Adj Low,Open_y,High_y,Low_y,Adj High_y,Adj Open_y,Adj Low_y,Adj Close_y,Volume_y
Date,,,,,,,,,,,,,,,,,
2010-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.620500,6.685500,6.483000,6.685500,6.620500,6.483000,6.517500,365702000.0
2010-03-17,6.567000,6.567000,6.634500,6.561000,6.620500,87176000.0,6.620500,6.634500,6.561000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-18,6.638000,6.638000,6.642500,6.522000,6.551000,100426000.0,6.551000,6.642500,6.522000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-19,6.517500,6.517500,6.685500,6.483000,6.685500,178100000.0,6.685500,6.685500,6.483000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.510000,6.849500,6.360000,6.849500,6.510000,6.360000,6.753000,741364000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-12,198.889999,198.889999,201.520004,195.289993,200.720001,43592200.0,200.720001,201.520004,195.289993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.720001,201.520004,195.289993,201.520004,200.720001,195.289993,198.889999,305145400.0
2025-03-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.720001,201.520004,195.289993,201.520004,200.720001,195.289993,198.889999,305145400.0


In [166]:
N = 10

In [167]:
df['tmp_col'] = np.where(df['SMA_5_day_Cross_SMA_20_day'] >0, 1, 0)
df[f'SMA_5_day_Cross_SMA_20_day_cumsum_{N}'] = df['tmp_col'].rolling(window=N, min_periods=1).sum()

In [168]:
df[['Date', 'SMA_5_day_Cross_SMA_20_day',f'SMA_5_day_Cross_SMA_20_day_cumsum_{N}', 'Adj Close', 'SMA_20_day']]

,Date,SMA_5_day_Cross_SMA_20_day,SMA_5_day_Cross_SMA_20_day_cumsum_10,Adj Close,SMA_20_day
0,2025-01-16,0,0.0,220.660004,223.045501
1,2025-01-17,0,0.0,225.940002,222.785001
2,2025-01-21,1,1.0,230.710007,223.294501
3,2025-01-22,0,1.0,235.009995,223.880502
4,2025-01-23,0,1.0,235.419998,224.405502
5,2025-01-24,0,1.0,234.850006,224.895002
6,2025-01-27,0,1.0,235.419998,225.213502
7,2025-01-28,0,1.0,238.149994,225.768501
8,2025-01-29,1,2.0,237.070007,226.434502
9,2025-01-30,0,2.0,234.639999,1000.000000


In [1]:
import pandas as pd

In [3]:
input_file = '/Users/liuda/Local/data/trading/data/download/20250318/YTB_KOL.csv'
SPY_input_file = '/Users/liuda/Local/data/trading/data/stocks/ticker/history//SPY.csv'

df = pd.read_csv(input_file)
df = df.sort_values(by=['publish_date']).reset_index(drop=True)
SPY_trading_date = pd.read_csv(SPY_input_file)[['Date']]

In [4]:
df['publish_date'] = pd.to_datetime(df['publish_date'], format='%Y-%m-%d', errors='coerce')
SPY_trading_date['Date'] = pd.to_datetime(SPY_trading_date['Date'], format='%Y-%m-%d', errors='coerce')
df = pd.merge(df, SPY_trading_date, left_on='publish_date', right_on='Date', how='outer').reset_index(drop=True)
df['trading_date'] = df['Date'].ffill()  # 线上使用
df = df.dropna(subset=['publish_date'])


In [6]:
df.to_csv('/Users/liuda/Desktop/1.csv', index=False)